In [2]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from Two_Class_MNIST import TwoClassMnist as tcm

Using TensorFlow backend.


In [3]:
x_train, x_test, y_train, y_test, input_shape = tcm.returnDataSet(1, 2, 28)
dataset = ((x_train, y_train), (x_test, y_test))
RUN_NAME = "rsmsw"
num_classes = 2
batch_size = 128
epochs = 1

x_train shape: (12700, 28, 28, 1)
12700 train samples
2167 test samples


In [4]:
def make_model(dense_layer_sizes, filters, kernel_size, pool_size):
    '''Creates model comprised of 2 convolutional layers followed by dense layers

    dense_layer_sizes: List of layer sizes.
        This list has one number for each layer
    filters: Number of convolutional filters in each convolutional layer
    kernel_size: Convolutional kernel size
    pool_size: Size of pooling area for max pooling
    '''

    model = Sequential()
    model.add(Conv2D(filters, kernel_size,
                     padding='valid',
                     input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(filters, kernel_size))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))

    model.add(Flatten())
    for layer_size in dense_layer_sizes:
        model.add(Dense(layer_size))
        model.add(Activation('relu'))
        
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    return model

In [5]:
dense_size_candidates = [[32], [64], [32, 32], [64, 64]]
my_classifier = KerasClassifier(make_model, batch_size=32)
# Embed in to dictionary
param_dist = {"batch_size": [8, 16],
              "epochs": [1, 2],
              "filters": [8],
              "kernel_size": [3],
              "pool_size": [2],
              "dense_layer_sizes": dense_size_candidates}


In [8]:
validator = RandomizedSearchCV(my_classifier,
                               param_distributions=param_dist,
                               scoring='neg_log_loss',
                               n_jobs=1)
history = validator.fit(x_train, y_train)

print('The parameters of the best model are: ')
print(validator.best_params_)

Epoch 1/2
8466/8466 [==============================] - 12s 1ms/step - loss: 0.0640 - acc: 0.9807
Epoch 2/2
8466/8466 [==============================] - 11s 1ms/step - loss: 0.0298 - acc: 0.9903
Epoch 1/2
8467/8467 [==============================] - 13s 1ms/step - loss: 0.0632 - acc: 0.9805
Epoch 2/2
8467/8467 [==============================] - 12s 1ms/step - loss: 0.0285 - acc: 0.9917
Epoch 1/2
8467/8467 [==============================] - 13s 2ms/step - loss: 0.0731 - acc: 0.9786
Epoch 2/2
8467/8467 [==============================] - 12s 1ms/step - loss: 0.0358 - acc: 0.9901
Epoch 1/1
8466/8466 [==============================] - 13s 2ms/step - loss: 0.0735 - acc: 0.9790
Epoch 1/1
8467/8467 [==============================] - 12s 1ms/step - loss: 0.0906 - acc: 0.9734
Epoch 1/1
8467/8467 [==============================] - 11s 1ms/step - loss: 0.0738 - acc: 0.9802
Epoch 1/2
8466/8466 [==============================] - 16s 2ms/step - loss: 0.0685 - acc: 0.9797
Epoch 2/2
8466/8466 [=========

In [1]:

# validator.best_estimator_ returns sklearn-wrapped version of best model.
# validator.best_estimator_.model returns the (unwrapped) keras model
best_model = validator.best_estimator_.model
metric_names = best_model.metrics_names
metric_values = best_model.evaluate(x_test, y_test)
for metric, value in zip(metric_names, metric_values):
    print(metric, ': ', value)

NameError: name 'validator' is not defined

In [11]:
import matplotlib.pyplot as plt
# Get training and test accuracy histories
training_accuracy = history['acc']
test_accuracy = history['val_acc']

# Create count of the number of epochs
epoch_count = range(1, len(training_accuracy) + 1)

# Visualize accuracy history
plt.plot(epoch_count, training_accuracy, 'r--')
plt.plot(epoch_count, test_accuracy, 'b-')
plt.legend(['Training Accuracy', 'Test Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy Score')
plt.show();


# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

TypeError: 'RandomizedSearchCV' object is not subscriptable